In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import json
from collections import Counter
import collections
import pickle
from termcolor import colored

import os
import time
import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as data_utils

from torchtext.vocab import Vocab, Vectors


from sklearn.metrics import accuracy_score, matthews_corrcoef, f1_score

/home/hyuns6100/anaconda3/envs/MentalHealth/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
###############################################################################
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

print(">>>>>>>>>>> CUDA available :", torch.cuda.is_available())
device = 'cuda' if torch.cuda.is_available() else 'cpu'
###############################################################################

>>>>>>>>>>> CUDA available : True


In [51]:
# # GPU 사용 가능 -> True, GPU 사용 불가 -> False
# print(torch.cuda.is_available())
# tensor = torch.rand(3, 3).cuda()
# print(tensor)

## Config 구성

In [3]:
import easydict

args = easydict.EasyDict({
    'data_path' : "/home/hyuns6100/Mental-Heatlh-Care/data/dailydialog_conv35seq_splits.json", # 데이터 경로
    'result_path': "/home/hyuns6100/Mental-Heatlh-Care/Result/",
    'result_text_path': "/home/hyuns6100/Mental-Heatlh-Care/Result/result_text/",
    'best_result_path': "/home/hyuns6100/Mental-Heatlh-Care/Result/best_results.pkl",
    'wv_path': "/home/hyuns6100/Mental-Heatlh-Care/data/", 
    'word_vector':"wiki-news-300d-1M.vec",
    
    'lr': 1e-3,
    'batch_size':32,
    'train_epochs':1000,
    'n_classes': 7,
    'n_train_class': 7, 
	'n_val_class': 7, 
	'n_test_class': 7, 
    'labels': [1, 2, 3, 4, 5, 6],
 
    'cnn_num_filters': 100,
    'cnn_filter_sizes': [3,4,5],
    'context_size': 35,
    'maxtokens': 30,
    'mlp_hidden': [300,300],
    'dropout': 0.1,
    
    'seed':330,
    'patience_metric': 'f1_micro',
    'finetune_ebd': False,
    'patience': 30,
    'save': True,
    'authors':False,
    'convmode': 'seq',
    'embedding': 'cnn',
    'classifier': 'mlp'
    })

In [4]:
def set_seed(seed):
    """
        Setting random seeds
    """
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)

## 데이터 토큰화 (이미 처리됨)


In [5]:
# def creaDailyDialogSeq():
#     	print(colored('CREATING DAILYDIALOG UNIFIED PREPROCESSED FILE FROM data/ijcnlp_dailydialog/  first make sure the per split dailydialog json files are there. Otherwise, please download dailydialog and run the formatting script as follows:', 'yellow'))
# 	print("""cd data/ijcnlp_dailydialog
# 	python3 parser_gg.py -i data/ijcnlp_dailydialog/train -o data/train 
# 	python3 parser_gg.py -i data/ijcnlp_dailydialog/validation -o data/validation
# 	python3 parser_gg.py -i data/ijcnlp_dailydialog/test -o data/test""")
# 	splits = {'train':'data/ijcnlp_dailydialog/train/dailydialog_train.json', 'test':'data/ijcnlp_dailydialog/test/dailydialog_test.json', 'val': 'data/ijcnlp_dailydialog/validation/dailydialog_validation.json'}
# 	splits_emotionflows_fp = { k: os.path.join( os.path.dirname(v), 'dailydialog_{}_emotionflow.json'.format(k) ) for k,v in splits.items() }
# 	txt2l = { 'no emotion': 0, 'anger': 1, 'disgust': 2, 'fear': 3, 'happiness': 4, 'sadness': 5, 'surprise': 6 }
# 	l2txt = { v:k for k, v in txt2l.items() } 
# 	emotionSet = list(set(list(txt2l.keys())))

# 	emotionFlows = {'train':[],'test':[],'val':[]}
# 	def getEmotionFlows(row, split):
# 		row = json.loads(row)
# 		emotionFlow = [ l2txt[r['label']] for r in row ]
# 		emotionFlows[split].append(emotionFlow)

# 	for k,v in tqdm(splits.items()):
# 		for row in tqdm( open(v, 'r').read().split('\n'), desc=colored(v, 'cyan') ) :
# 			getEmotionFlows(row, k)

# 	seq_lengths = np.array([ len(x) for x in emotionFlows['train'] ] )
# 	seq_size = np.percentile( seq_lengths, 90)
# 	# seq_size = max(seq_lengths)
# 	print(colored('seq_size', 'yellow'), seq_size, '75%:', np.percentile(seq_lengths, 75), 'max:', max(seq_lengths))

# 	def tokenize_seq(chat):
# 		tokenizer = TweetTokenizer()
# 		text =  [ ' '.join( list( map(lambda x: x.lower(), tokenizer.tokenize(m)) ) ) for m in chat['text'] ]
# 		chat['text'] = text
# 		return chat

# 	def _trimpad(size, row, pad=True, trim=True):
# 		''' trim and padding (with <pad>) '''
# 		if len(row['texts']) > size:
# 			row['texts'] = row['texts'][int(-size):]
# 			row['labels'] = row['labels'][int(-size):]
# 		else:
# 			row['texts'] = [ ['<pad>' for j in range(5)] for k in range( int(size) - len(row['texts'])  ) ]   +   row['texts']
# 			row['labels'] = [ 0 for k in range( int(size) - len(row['labels'])  ) ]   +   row['labels']
		
# 		assert len(row['texts']) == size
# 		assert len(row['labels']) == size
# 		return row

# 	dataSplits = {'train':[], 'test': [], 'val':[]}
# 	for split in dataSplits.keys():
# 		for i, data in tqdm( enumerate( [ json.loads(line) for line in open(splits[split], 'r').read().split('\n') ]  ), desc=colored('formatting sequence '+split, 'cyan'), total=len(emotionFlows[split]) ):
# 			# data = [ tokenize_seq(chat) for chat in tqdm(data, total=len(data), desc='tokenizing') ]
# 			data = [ tokenize_seq(chat) for chat in data ]
# 			entry = {'texts': [ x['text'] for x in data], 'labels': [ x['label'] for x in data], 'split': split}
# 			entry = _trimpad(seq_size, entry)
# 			dataSplits[split].append(entry)
	
	
# 	labels_train = Counter([ label for line in dataSplits['train'] for label in line['labels'] ])
# 	labels_val = Counter([ label for line in dataSplits['val'] for label in line['labels'] ])
# 	labels_test = Counter([ label for line in dataSplits['test'] for label in line['labels'] ])
# 	print(colored('labels balance', 'yellow'), labels_train, len(labels_train.keys()), labels_val, len(labels_val.keys()), labels_test, len(labels_test.keys()))

# 	records = dataSplits['train'] + dataSplits['val'] + dataSplits['test']
# 	jsonLines = [json.dumps(line) for line  in tqdm(records)]
# 	with open('data/dailydialog_conv{}seq_splits.json'.format(str(int(seq_size))), 'w') as f: f.write('\n'.join(jsonLines))
# 	print( colored('data/dailydialog_conv%sseq_splits.json created!' % (str(int(seq_size))), 'green') )
# 	print(colored('You can now run the labelling tasks.', 'green'))

## DataLoader 구성

In [5]:
def _get_dailydialog_seq_classes():
    '''
        seq consider no emotion label
        @return list of classes associated with each split
    '''
    label_dict = { 
        'no emotion': 0, 
        'anger': 1, 
        'disgust': 2,
        'fear': 3,
        'happiness': 4,
        'sadness': 5,
        'surprise': 6
    }

    train_classes = [0, 1, 2, 3, 4, 5, 6]
    val_classes = [0, 1, 2, 3, 4, 5, 6]
    test_classes = [0, 1, 2, 3, 4, 5, 6]

    return train_classes, val_classes, test_classes

def _load_json_seq(path, args):
    '''
        load data file
        @param path: str, path to the data file
        @return data: list of examples
    '''
    label = {}
    text_len = []
    with open(path, 'r', errors='ignore') as f:
        data = []
        for i, line in enumerate(f):
            row = json.loads(line)

            # count the number of examples per label
            for l in row['labels']:
                if int(l) not in label: label[int(l)] = 1
                else: label[int(l)] += 1

            item = {
                'id': i+1,
                'label': [int(r) for r in row['labels'] ],
                # 'text': [ r[:args['maxtokens']] for r in row['texts'] ]  # 30 # 50 # 80 truncate the text to 500 tokens
                'text': [ r[-args['maxtokens']:] for r in row['texts'] ]  # 30 # 50 # 80 truncate the text to the last tokens
            }

            if args.authors:
                item.update({'authors': [ int(a) for a in row['authors'] ]})

            if 'split' in row: item['split'] = row['split']

            text_len.append(len(row['texts']))

            data.append(item)

        #tprint('Class balance (load_json_seq):')

        print(label)

        print('Avg len: {}'.format(sum(text_len) / (len(text_len))))
        print('Max len: {}'.format(max(text_len)))

        return data
    
def _read_words(data, convmode=None):
    '''
        Count the occurrences of all words
        @param convmode: str, None for non conversational scope, 'naive' for classic or naive approach, 'conv' for conversation depth into account (one additional dim and nested values)
        @param data: list of examples
        @return words: list of words (with duplicates)
    '''
    words = []
    if convmode is None:
        for example in data:
            words += example['text']
    else:
        for example in data:
            for m in example['text']: 
                words += m     
    
    return words

def _meta_split_by_field(all_data, train_classes, val_classes, test_classes, seqmode=False):
    '''
        Split the dataset according to the specified train_classes, val_classes
        and test_classes
        Consider a 'split' field for the different train test val sets

        seqmode is a special mode to ensure sequences of labels to be taken into account

        @param all_data: list of examples (dictionaries)
        @param train_classes: list of int
        @param val_classes: list of int
        @param test_classes: list of int
        @param seqmode: bool 

        @return train_data: list of examples
        @return val_data: list of examples
        @return test_data: list of examples
    '''
    train_data, val_data, test_data = [], [], []

    if seqmode:
        for example in all_data:
            if example['split'] == 'train' and len(set(example['label']) & set(train_classes)) > 0: train_data.append(example)
            if example['split'] == 'val' and len(set(example['label']) & set(val_classes)) > 0: val_data.append(example)
            if example['split'] == 'test' and len(set(example['label']) & set(test_classes)) > 0: test_data.append(example)
    else: 
        for example in all_data:
            if example['split'] == 'train' and example['label'] in train_classes: train_data.append(example)
            if example['split'] == 'val' and example['label'] in val_classes: val_data.append(example)
            if example['split'] == 'test' and example['label'] in test_classes: test_data.append(example)

    return train_data, val_data, test_data

def _del_by_idx(array_list, idx, axis):
    '''
        Delete the specified index for each array in the array_lists

        @params: array_list: list of np arrays
        @params: idx: list of int
        @params: axis: int

        @return: res: tuple of pruned np arrays
    '''
    if type(array_list) is not list:
        array_list = [array_list]

    # modified to perform operations in place
    for i, array in enumerate(array_list):
        array_list[i] = np.delete(array, idx, axis)

    if len(array_list) == 1:
        return array_list[0]
    else:
        return array_list
    
    
def _data_to_nparray(data, vocab, args):
    '''
        Convert the data into a dictionary of np arrays for speed.
    '''
    doc_label = np.array([x['label'] for x in data], dtype=np.int64)

    raw = np.array([e['text'] for e in data], dtype=object)

    # compute the max text length
    text_len = np.array([len(m) for e in data for m in e['text']])
    max_text_len = max(text_len)
    seq_len = np.array(  [len(e['text']) for e in data]  )
    max_seq_len =  max(seq_len)
    ids = np.array([e['id'] for e in data])

    # initialize the big numpy array by <pad>
    text = vocab.stoi['<pad>'] * np.ones([len(data), max_seq_len, max_text_len], dtype=np.int64)
    
    del_idx = []
    # convert each token to its corresponding id
    for i in tqdm(range(len(data)), desc='converting tokens to ids'): # 모든 대화를 돌면서 한 대화 뭉텅이씩 처리
        for idx_x, x in enumerate(data[i]['text']): # 대화 내 모든 문장을 돌면서 한 문장씩 처리
                for idx_message, message in enumerate(x): # 각 문장 내 모든 토큰을 돌면서 각 토큰별 vocab을 이용하여 id로 변환
                        if message in vocab.stoi:
                                text[i, idx_x, idx_message] = vocab.stoi[message]
                        else:
                                text[i, idx_x, idx_message] = vocab.stoi['<unk>']
            # try:
            #     for idx_message, message in enumerate(x):
            #         text[i, idx_x, :len(message)] = [
            #                             vocab.stoi[token] if token in vocab.stoi else vocab.stoi['<unk>'] 
            #                             for token in message
            #                             ]
            # except Exception as e:
            #     print(e)
            #     print(x, idx_x)
            #     exit()

        # filter out document with only unk and pad
        if np.max(text[i]) < 2:
            del_idx.append(i)

    vocab_size = vocab.vectors.size()[0]
    
    print("del_idx: ", del_idx) # 빈 리스트 반환됨
    
    ## Curation for padding (string instead of list of list)
    raw = [ ["<pad>" if m == ["<pad>", "<pad>", "<pad>", "<pad>", "<pad>"] else m for m in c ] for c in raw ]

    if args.authors:
        # trim and pad authors (should have been done in dtaa creation but left here for comparison purposes)
        authors = list()
        for x in data:
            a = len(x['authors'])
            if a < args.context_size: 
                authors.append(x['authors'] + [0 for i in range(18-a)])
            elif a > args.context_size:
                authors.append( x['authors'][int(-args.context_size):] )
            else:
                authors.append(x['authors'])
        authors = np.array(authors, dtype=np.int64)

        ids, text_len, text, doc_label, raw, authors = _del_by_idx(
                [ids, text_len, text, doc_label, raw, authors], del_idx, 0)
        new_data = {
            'ids': ids,
            'text': text,
            'text_len': text_len,
            'label': doc_label,
            'raw': raw,
            'authors': authors,
            'vocab_size': vocab_size,
        }
        
    else: ## authors = False
        #ids, text_len, text, doc_label, raw = _del_by_idx( [ids, text_len, text, doc_label, raw], del_idx, 0)
        new_data = {
            'ids': ids,
            'text': text,
            'text_len': text_len,
            'label': doc_label,
            'raw': raw,
            'vocab_size': vocab_size,
        }
    return new_data
    

In [6]:
class dailydialog_DataLoader:
    def __init__(self, args):
        self.args = args
        
    def load_dataset(self):
        train_classes, val_classes, test_classes = _get_dailydialog_seq_classes()
        assert(len(train_classes) == args.n_train_class)
        assert(len(val_classes) == args.n_val_class)
        assert(len(test_classes) == args.n_test_class)
        
        all_data = _load_json_seq(self.args.data_path, self.args)
        
        # Loading word vector
        path = os.path.join(self.args.wv_path, self.args.word_vector)
        if not os.path.exists(path):
            # Download the word vector and save it locally:
            print('Downloading word vectors')
            import urllib.request
            urllib.request.urlretrieve(
                'https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.vec',
                path)
        
        vectors = Vectors(args.word_vector, cache=args.wv_path)
        min_freq = 2
        vocab = Vocab(collections.Counter(_read_words(all_data, convmode=args.convmode)), vectors=vectors,
                  specials=['<pad>', '<unk>'], min_freq=min_freq)
        
         # print word embedding statistics
        wv_size = vocab.vectors.size()
        print('Total num. of words: {}, word vector dimension: {}'.format(
            wv_size[0],
            wv_size[1]))
        
        num_oov = wv_size[0] - torch.nonzero(
            torch.sum(torch.abs(vocab.vectors), dim=1)).size()[0]
        print(('Num. of out-of-vocabulary words'
              '(they are initialized to zeros): {}').format( num_oov))
        
        # Split into meta-train, meta-val, meta-test data (or just splits)
        train_data, val_data, test_data = _meta_split_by_field(all_data, train_classes ,val_classes, test_classes, seqmode=True)
        trainset = Counter([l for d in train_data for l in d['label']])
        valset = Counter([l for d in val_data for l in d['label']])
        testset = Counter([l for d in test_data for l in d['label']])
        print(colored('check sets splits', 'yellow'), trainset, len(list(trainset.keys())), valset, len(list(valset.keys())),  testset, len(list(testset.keys())))
        
        # Convert everything into np array for fast data loading
        train_data = _data_to_nparray(train_data, vocab, args)
        val_data = _data_to_nparray(val_data, vocab, args)
        test_data = _data_to_nparray(test_data, vocab, args)

        train_data['is_train'] = True
        
        #return converted_train_data, converted_val_data, converted_test_data, train_data, val_data, test_data, vocab
        return train_data, val_data, test_data, vocab

In [7]:
set_seed(args.seed)
loader = dailydialog_DataLoader(args)
#converted_train_data, converted_val_data, converted_test_data, train_data, val_data, test_data, vocab = loader.load_dataset()
train_data, val_data, test_data, vocab = loader.load_dataset()

{0: 441723, 4: 12885, 6: 1823, 3: 174, 2: 353, 5: 1150, 1: 1022}
Avg len: 35.0
Max len: 35
Total num. of words: 13967, word vector dimension: 300
Num. of out-of-vocabulary words(they are initialized to zeros): 1947
check sets splits Counter({0: 374103, 4: 11182, 6: 1600, 5: 969, 1: 827, 2: 303, 3: 146}) 7 Counter({0: 34039, 4: 684, 6: 107, 5: 79, 1: 77, 3: 11, 2: 3}) 7 Counter({0: 33581, 4: 1019, 1: 118, 6: 116, 5: 102, 2: 47, 3: 17}) 7


converting tokens to ids: 100%|██████████| 11118/11118 [00:01<00:00, 8203.64it/s]


del_idx:  []


converting tokens to ids: 100%|██████████| 1000/1000 [00:00<00:00, 9460.76it/s]


del_idx:  []


converting tokens to ids: 100%|██████████| 1000/1000 [00:00<00:00, 9086.17it/s]

del_idx:  []


In [100]:
# import pickle

# # 파일로 저장
# with open('/home/hyuns6100/Mental-Heatlh-Care/onnx/vocab.pkl', 'wb') as f:
#     pickle.dump(vocab, f)
    
# # 파일 불러오기
# with open('/home/hyuns6100/Mental-Heatlh-Care/onnx/vocab.pkl', 'rb') as f:
#     vocab = pickle.load(f)

In [8]:
from copy import copy

copy_test_data = test_data.copy()

In [59]:
copy_test_data['text'].shape

(1000, 35, 30)

In [60]:
# ## 토큰화 예시 확인
# data = train_data
# text_len = np.array([len(m) for e in data for m in e['text']])
# max_text_len = max(text_len)
# seq_len = np.array(  [len(e['text']) for e in data]  )
# max_seq_len =  max(seq_len)
# ids = np.array([e['id'] for e in data])


In [61]:
# text = vocab.stoi['<pad>'] * np.ones([len(data), max_seq_len, max_text_len], dtype=np.int64)

In [62]:
# for i in tqdm(range(len(data)), desc='converting tokens to ids'): # 모든 대화를 돌면서 한 대화 뭉텅이씩 처리
#         for idx_x, x in enumerate(data[i]['text']): # 대화 내 모든 문장을 돌면서 한 문장씩 처리
#                 for idx_message, message in enumerate(x): # 각 문장 내 모든 토큰을 돌면서 각 토큰별 vocab을 이용하여 id로 변환
#                         print(message)
#                         if message in vocab.stoi:
#                                 text[i, idx_x, idx_message] = vocab.stoi[message]
#                         else:
#                                 text[i, idx_x, idx_message] = vocab.stoi['<unk>']
#                 break

In [63]:
# text.shape # i번째 대화 뭉텅이, i번째 문장, i번째 토큰

In [64]:
# doc_label = np.array([x['label'] for x in data], dtype=np.int64)

# raw = np.array([e['text'] for e in data], dtype=object)

In [65]:
# doc_label.shape

In [66]:
# doc_label[0]

In [67]:
# text

In [68]:
# text_len = np.array([len(m) for e in data for m in e['text']])
# max_text_len = max(text_len)
# seq_len = np.array(  [len(e['text']) for e in data]  )
# max_seq_len =  max(seq_len)
# ids = np.array([e['id'] for e in data])

# # initialize the big numpy array by <pad>
# text = vocab.stoi['<pad>'] * np.ones([len(data), max_seq_len, max_text_len], dtype=np.int64)

# del_idx = []
# # convert each token to its corresponding id
# for i in tqdm(range(len(data)), desc='converting tokens to ids'):
#     for idx_x, x in enumerate(data[i]['text']):
#         for idx_message, message in enumerate(x):
#                 text[i, idx_x, :len(message)] = [
#                                     vocab.stoi[token] if token in vocab.stoi else vocab.stoi['<unk>'] 
#                                     for token in message
#                                     ]

In [9]:
class SupervisedDataset(Dataset):
    def __init__(self, data, args):
        '''
            data : dict_keys(['ids', 'text', 'text_len', 'label', 'raw', 'vocab_size', 'is_train']) 'authors'
        '''
        # self.berttokenizer = berttokenizer
        # if self.berttokenizer:
        #     self.tokenizer = AutoTokenizer.from_pretrained(os.path.join(args.pretrained_bert))
        self.args = args
        self.ids = data['ids']
        self.text = data['text']
        self.text_len = data['text_len']
        self.label = data['label']
        self.raw = data['raw']
        #self.authors = data['authors']
        self.vocab_size = data['vocab_size']
        self.train = False
        if 'is_train' in data:
            self.is_train = data['is_train']
            self.train = True

    def __len__(self):
        return len(self.ids)
    
    def __getitem__(self, idx):
        
        item = {
            'ids': self.ids[idx], 
            'text': self.text[idx], 
            'text_len': self.text_len[idx], 
            # 'label': np.expand_dims(self.label[idx],0),  # .expand_dims(x, axis=0) unsqueeze(0) for seq labelling (bert)
            'label': self.label[idx],
            # 'raw': self.raw[idx].tolist(), 
            'vocab_size': self.vocab_size,
            #'authors': self.authors[idx]
        }

       
        if self.train: item.update({'is_train': self.is_train})
        
        return item

In [10]:
train_loader = data_utils.DataLoader(SupervisedDataset(train_data, args), batch_size=args.batch_size, num_workers=2, shuffle=False)
val_loader = data_utils.DataLoader(SupervisedDataset(val_data, args), batch_size=args.batch_size, num_workers=2, shuffle=False)
test_loader = data_utils.DataLoader(SupervisedDataset(test_data, args), batch_size=args.batch_size, num_workers=2, shuffle=False)

## Embedding

In [11]:
class WORDEBD(nn.Module):
    '''
        An embedding layer that maps the token id into its corresponding word
        embeddings. The word embeddings are kept as fixed once initialized.
    '''
    def __init__(self, vocab, finetune_ebd):#, specific_vocab_size=None):
        super(WORDEBD, self).__init__()

        self.vocab_size, self.embedding_dim = vocab.vectors.size()
        # if specific_vocab_size != None: self.vocab_size = specific_vocab_size
        self.embedding_layer = nn.Embedding(
                self.vocab_size, self.embedding_dim)
        self.embedding_layer.weight.data = vocab.vectors

        self.finetune_ebd = finetune_ebd

        if self.finetune_ebd:
            self.embedding_layer.weight.requires_grad = True
        else:
            self.embedding_layer.weight.requires_grad = False

    def forward(self, data, weights=None):
        '''
            @param text: batch_size * max_text_len
            @return output: batch_size * max_text_len * embedding_dim
        '''
        if (weights is None): #or (self.finetune_ebd == False):
            return self.embedding_layer(data['text'])

        else:
            return F.embedding(data['text'],
                               weights['ebd.embedding_layer.weight'])

class CNNseq(nn.Module):
    '''
        An aggregation method that encodes every document through different
        convolution filters (followed by max-over-time pooling).
    '''
    def __init__(self, ebd, args):
        super(CNNseq, self).__init__()
        self.args = args

        self.ebd = ebd # pre-trained FastText로 initialization된 token representation => WORDEBD => nn.Embedding layer로 매핑된 것

        self.input_dim = self.ebd.embedding_dim

        # Convolution
        self.convs = nn.ModuleList([nn.Conv1d(
                    in_channels=self.input_dim,
                    out_channels=args.cnn_num_filters,
                    kernel_size=K) for K in args.cnn_filter_sizes])
        
        self.relu = nn.ReLU()

        self.ebd_dim = args.cnn_num_filters * len(args.cnn_filter_sizes)

    def _conv_max_pool(self, x, conv_filter=None, weights=None):
        '''
        Compute sentence level convolution
        Input:
            x:      batch_size, max_doc_len, embedding_dim
        Output:     batch_size, num_filters_total
        '''
        assert(len(x.size()) == 3) # [batch_size==max_sentences, max_tokens, embedding_dim]

        x = x.permute(0, 2, 1)  # batch_size, embedding_dim, doc_len
        x = x.contiguous()

        # Apply the 1d conv. Resulting dimension is
        # [batch_size, num_filters, doc_len-filter_size+1] * len(filter_size)
        assert(not ((conv_filter is None) and (weights is None)))
        if conv_filter is not None:
            x = [conv(x) for conv in conv_filter]

        # elif weights is not None:
        #     x = [F.conv1d(x, weight=weights['convs.{}.weight'.format(i)],
        #                 bias=weights['convs.{}.bias'.format(i)])
        #         for i in range(len(self.args.cnn_filter_sizes))]

        ## max pool over time. Resulting dimension is
        ## [batch_size, num_filters] * len(filter_size)
        #x = [F.max_pool1d(sub_x, sub_x.size(2)).squeeze(2) for sub_x in x]
        
        ## nn.MaxPool1d로 다시 생성
        max_pooled_outputs = []
        for sub_x in x:
            pool_size = sub_x.size(2)
            max_pool = nn.MaxPool1d(pool_size) # output shape: [batch_size, num_filters, 1]
            pooled = max_pool(sub_x).squeeze(2) # output shape: [batch_size, num_filters]
            max_pooled_outputs.append(pooled)
        
        # concatenate along all filters. Resulting dimension is
        # output: [batch_size, num_filters_total]
        x = torch.cat(max_pooled_outputs, 1) # output shape: [batch_size, num_filters*3]
        #x = torch.cat(x, 1)
        x = self.relu(x) #F.relu(x)
        return x

    def forward(self, data, weights=None):
        '''
            @param data dictionary
                @key text: batch_size * max_text_len
            @param weights placeholder used for maml

            @return output: batch_size * embedding_dim
        '''

        device = data['text'].device
        
        # Apply the word embedding, result:  batch_size, doc_len, embedding_dim
        
        ebd = self.ebd(data, weights) # ouptut: [batch_size, max_sentences, max_tokens, embedding_dim]

        # apply 1d conv + max pool, result:  batch_size, num_filters_total        
        ref = tuple(data['text'].size())
        shape = (ref[0], ref[1], ( len(self.args.cnn_filter_sizes) * self.args.cnn_num_filters))
        output = torch.randn(shape).to(device)
        
        if weights is None:
            for i in range(ebd.size(0)): # 각 배치에 대해
                out = self._conv_max_pool(ebd[i], conv_filter=self.convs) # 각 문장에 대해 처리 => (35, 300)
                output[i] = out
        
        else:
            for i in range(ebd.size(0)):
                out = self._conv_max_pool(ebd[i], weights=weights)
                output[i] = out
        
        return output
    
def get_embedding(vocab, args):
    ebd = WORDEBD(vocab, args.finetune_ebd)
    model = CNNseq(ebd, args)
    model.to(device)
    return model

In [13]:
# # output = out_XS.view(-1, args.n_classes)  # new shape: [32*35, 7]
# # target = YS.view(-1)  # new shape: [32*35]

# output = torch.rand((32*35, 7))
# target = test_data['label'][:32].view()

# output.shape, target.shape

In [14]:
# for batch in test_loader:
#     YS = batch['label']
#     break

In [53]:
# YS = batch['label']
# target = YS.view(-1)
# target.shape

torch.Size([1120])

In [54]:
# torch.mean((torch.argmax(output, dim=1) == target).float()).item()

0.15267856419086456

In [64]:
# torch.mean((torch.argmax(output, dim=1) == target).float())

tensor(0.1527)

In [66]:
# (torch.argmax(output, dim=1) == target).float().sum() / len(output)

tensor(0.1527)

In [12]:
class distLinear(nn.Module):
    def __init__(self, indim, outdim):
        super(distLinear, self).__init__()
        self.L = nn.Linear(indim, outdim, bias = False)
        # split the weight update component to direction and norm
        # WeightNorm.apply(self.L, 'weight', dim=0)

        # a fixed scale factor to scale the output of cos value
        # into a reasonably large input for softmax
        self.scale_factor = 10

    def forward(self, x):

        x_norm = torch.norm(x, p=2, dim =1).unsqueeze(1).expand_as(x)
        x_normalized = x.div(x_norm + 0.00001)
        # L_norm = torch.norm(self.L.weight.data, p=2, dim=1).unsqueeze(1).expand_as(self.L.weight.data)

        # self.L.weight.data = self.L.weight.data.div(L_norm + 0.00001)

        cos_dist = self.L(x_normalized)  # matrix product by forward function
        scores = self.scale_factor * (cos_dist)

        return scores
    
class BASE(nn.Module):
    '''
        BASE model
    '''
    def __init__(self, args):
        super(BASE, self).__init__()
        self.args = args

        # cached tensor for speed
        # self.I_way = nn.Parameter(torch.eye(self.args.way, dtype=torch.float),
        #                           requires_grad=False)
    
    @staticmethod
    def compute_acc(pred, true, dim=1, nomax=False):
        '''
            Compute the accuracy.
            @param pred: batch_size * num_classes
            @param true: batch_size
        '''
        if nomax: return torch.mean((pred == true).float()).item()
        else: return torch.mean((torch.argmax(pred, dim=dim) == true).float()).item()
        
    @staticmethod
    def compute_f1(y_pred, true, dim=1, nomax=False,  labels=None, average='weighted'):
        '''
            Compute the weighted f1 score.
            @param pred: batch_size * num_classes
            @param true: batch_size
        '''
        if not nomax: _, y_pred = torch.max(y_pred, dim)

        f1 = f1_score(true.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), average=average, labels=labels)

        return f1

    @staticmethod
    def compute_mcc(y_pred, true, dim=1, nomax=False):
        '''
            Compute the matthews correlation coeficient.
            @param pred: batch_size * num_classes
            @param true: batch_size
        '''
        if not nomax: _, y_pred = torch.max(y_pred, dim)

        mcc = matthews_corrcoef(true.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

        return mcc
    
    @staticmethod
    def compute_f1_micro_noneutral(y_pred, true, dim=1, nomax=False, labels=None):
        
        if not nomax: _, y_pred = torch.max(y_pred, dim)

        f1 = f1_score(true.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), average='micro', labels=labels)

        return f1

class MLPseq(BASE):
    def __init__(self, ebd_dim, args, top_layer=None):
        super(MLPseq, self).__init__(args)

        self.args = args
        self.ebd_dim = ebd_dim

        self.mlp = self._init_mlp(ebd_dim, self.args.mlp_hidden, self.args.dropout)
        self.out = self.get_top_layer(self.args, self.args.n_classes)
        #self.top_layer = top_layer
        self.dropout = nn.Dropout(self.args.dropout)

    @staticmethod
    def get_top_layer(args, n_classes):
        '''
            Creates final layer of desired type
            @return final classification layer
        '''
        return nn.Linear(args.mlp_hidden[-1], n_classes)

        
    def _init_mlp(self, in_d, hidden_ds, drop_rate):
        modules = []

        for d in hidden_ds[:-1]:
            modules.extend([
                nn.Dropout(drop_rate),
                nn.Linear(in_d, d),
                nn.ReLU()])
            in_d = d

        modules.extend([
            nn.Dropout(drop_rate),
            nn.Linear(in_d, hidden_ds[-1])])

        return nn.Sequential(*modules)
    
    def forward(self, XS, YS=None, XQ=None, YQ=None, weights=None, return_preds=False):
        '''
            if y is specified, return loss and accuracy
            otherwise, return the transformed x

            @param: XS: batch_size * input_dim
            @param: YS: batch_size (optional)

            @return: XS: batch_size * output_dim
        '''

        # normal training procedure, train stage only use query
        # if weights is None:
        #     XS = self.mlp(XS)
        # else:
        #     # find weight and bias keys for the mlp module
        #     w_keys, b_keys = [], []
        #     for key in weights.keys():
        #         if key[:4] == 'mlp.':
        #             if key[-6:] == 'weight':
        #                 w_keys.append(key)
        #             else:
        #                 b_keys.append(key)

        #     for i in range(len(w_keys)-1):
        #         #XS = F.dropout(XS, self.args.dropout, training=self.training)
        #         XS = self.dropout(XS)
        #         XS = F.linear(XS, weights[w_keys[i]], weights[b_keys[i]])
        #         XS = F.relu(XS)

        #     XS = F.dropout(XS, self.args.dropout, training=self.training)
        #     XS = F.linear(XS, weights[w_keys[-1]], weights[b_keys[-1]])

        XS = self.mlp(XS)
        XS = self.out(XS) # output: [batch, max_sentence, n_class]
        
        # if self.top_layer is not None:
        #     XS = self.top_layer(XS)

        # # normal training procedure, compute loss/acc
        # if YS is not None:
        #     # if self.args.taskmode == 'episodic':
        #     #     ## useful for episodes, ignored for full supervised
        #     #     _, YS = torch.unique(YS, sorted=True, return_inverse=True)
        #     loss = F.cross_entropy(XS, YS)
        #     acc = BASE.compute_acc(XS, YS)
        #     f1 = BASE.compute_f1(XS, YS)
        #     mcc = BASE.compute_mcc(XS, YS)

        #     if return_preds:
        #         _, y_pred = torch.max(XS, dim=1)
        #         return acc, loss, f1, mcc, y_pred, YS
        #     else:
        #         return acc, loss, f1, mcc

        # else:
        #     return XS
        
        return XS

def get_classifier(emb_dim, args):
    model = MLPseq(emb_dim, args)
    model.to(device)
    return model

In [13]:
model = {}
model["ebd"] = get_embedding(vocab, args)
model['ebd'].train()
model["clf"] = get_classifier(model["ebd"].ebd_dim, args)

In [14]:
model["ebd"], model["clf"]

(CNNseq(
   (ebd): WORDEBD(
     (embedding_layer): Embedding(13967, 300)
   )
   (convs): ModuleList(
     (0): Conv1d(300, 100, kernel_size=(3,), stride=(1,))
     (1): Conv1d(300, 100, kernel_size=(4,), stride=(1,))
     (2): Conv1d(300, 100, kernel_size=(5,), stride=(1,))
   )
   (relu): ReLU()
 ),
 MLPseq(
   (mlp): Sequential(
     (0): Dropout(p=0.1, inplace=False)
     (1): Linear(in_features=300, out_features=300, bias=True)
     (2): ReLU()
     (3): Dropout(p=0.1, inplace=False)
     (4): Linear(in_features=300, out_features=300, bias=True)
   )
   (out): Linear(in_features=300, out_features=7, bias=True)
   (dropout): Dropout(p=0.1, inplace=False)
 ))

## Train

In [15]:
import itertools

def grad_param(model, keys):
    '''
        Return a generator that generates learnable parameters in
        model[key] for key in keys.
    '''
    if len(keys) == 1:
        return filter(lambda p: p.requires_grad,
                model[keys[0]].parameters())
    else:
        return filter(lambda p: p.requires_grad,
                itertools.chain.from_iterable(
                    model[key].parameters() for key in keys))

def get_norm(model):
    '''
        Compute norm of the gradients
    '''
    total_norm = 0

    for p in model.parameters():
        if p.grad is not None:
            p_norm = p.grad.data.norm()
            total_norm += p_norm.item() ** 2

    total_norm = total_norm ** 0.5

    return total_norm

In [16]:
def train(train_data, val_data, model, args, loader=None):
    '''
        Train the model
        Use val_data to do early stopping
    '''

    
    out_dir = args.result_path

    
    best_acc = 0
    best_score = 0
    sub_cycle = 0
    best_path = None

    # opt = torch.optim.Adam(grad_param(model, ['ebd', 'clf']), lr=args.lr)
    opt = torch.optim.Adam(grad_param(model, ['ebd', 'clf']), lr=args.lr, betas=(0.9, 0.98), eps=pow(10, -9)) # CESTa optimizer parameters


    #scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, 'max', patience=2, factor=0.1, verbose=True) # args.patience//2

    print("{}, Start training supervised".format(datetime.datetime.now().strftime('%02y/%02m/%02d %H:%M:%S')), flush=True)

    for ep in range(args.train_epochs):

        grad = {'clf': [], 'ebd': []}
        
        # train on training set
        for batch in tqdm(loader, ncols=80, leave=False, desc=colored('Training on train', 'yellow')):
            train_one(batch, model, opt, args, grad)

        # Evaluate validation accuracy
        # cur_acc, cur_std, cur_f1, cur_f1_std, cur_mcc, cur_mcc_std, cur_f1_micro, cur_f1_micro_std = test(val_data, model, args, False)
        cur_acc, cur_std, cur_f1, cur_f1_std, cur_mcc, cur_mcc_std, cur_f1_micro, cur_f1_micro_std = test(val_data, model, args, False, loader=val_loader)
        print(cur_acc, cur_std, cur_f1, cur_f1_std, cur_mcc, cur_mcc_std)
        
        print(("{}, {:s} {:2d}, {:s} {:s}{:>7.4f} ± {:>6.4f}, {:s}{:>7.4f} ± {:>6.4f}, {:s}{:>7.4f} ± {:>6.4f}, {:s}{:>7.4f} ± {:>6.4f},"
               "{:s} {:s}{:>7.4f}, {:s}{:>7.4f}").format(
               datetime.datetime.now().strftime('%02y/%02m/%02d %H:%M:%S'),
               "ep", ep,
               colored("val  ", "cyan"),
               colored("acc:", "blue"), cur_acc, cur_std,
               colored("f1:", "blue"), cur_f1, cur_f1_std,
               colored("mcc:", "blue"), cur_mcc, cur_mcc_std,
               colored("f1 micro:", "blue"), cur_f1_micro, cur_f1_micro_std,
               colored("train stats", "cyan"),
               colored("ebd_grad:", "blue"), np.mean(np.array(grad['ebd'])),
               colored("clf_grad:", "blue"), np.mean(np.array(grad['clf'])),
               ), flush=True)
        scores = {'acc':cur_acc, 'f1': cur_f1, 'mcc': cur_mcc, 'f1_micro': cur_f1_micro}

        # Update the current best model if val acc is better
        # if cur_acc > best_acc:
        #     best_acc = cur_acc
        if scores[args.patience_metric] > best_score:
            best_score = scores[args.patience_metric]
            best_path = os.path.join(out_dir, str(ep))

            print( colored( "{}, Attempt to save cur best model to {}".format(
                datetime.datetime.now().strftime('%02y/%02m/%02d %H:%M:%S'),
                best_path) , 'magenta' ))

            while True:
                try:
                    torch.save(model['ebd'].state_dict(), best_path + '.ebd')
                    torch.save(model['clf'].state_dict(), best_path + '.clf')
                    break
                except (FileNotFoundError):
                    continue
            
            # save current model
            print( colored( "{}, Saved cur best model to {}".format(
                datetime.datetime.now().strftime('%02y/%02m/%02d %H:%M:%S'),
                best_path) , 'magenta' ))

            sub_cycle = 0
        else:
            sub_cycle += 1

        #if args.scheduler: scheduler.step(cur_acc)

        # Break if the val acc hasn't improved in the past patience epochs
        if sub_cycle == args.patience:
            break

    print("{}, End of training. Restore the best weights".format(
            datetime.datetime.now().strftime('%02y/%02m/%02d %H:%M:%S')),
            flush=True)

    # restore the best saved model
    while True:
        try:
            model['ebd'].load_state_dict(torch.load(best_path + '.ebd'))
            model['clf'].load_state_dict(torch.load(best_path + '.clf'))
            break
        except (FileNotFoundError):
            continue
    

    if args.save:
        # save the current model
        # out_dir = os.path.abspath(os.path.join(
        #                               os.path.curdir,
        #                               "saved-runs",
        #                               str(int(time.time() * 1e7))))
        out_dir = args.result_text_path
        
        # if args.result_path != '':
        #     dir_path = os.path.split(args.result_path)[0]
        #     out_dir = os.path.abspath(os.path.join(
        #                               os.path.curdir, dir_path) )

        if not os.path.exists(out_dir):
            os.makedirs(out_dir)

        best_path = os.path.join(out_dir, 'best')

        print(colored("{}, Save best model to {}".format(
            datetime.datetime.now().strftime('%02y/%02m/%02d %H:%M:%S'),
            best_path), "green"), flush=True)

        torch.save(model['ebd'].state_dict(), best_path + '.ebd')
        torch.save(model['clf'].state_dict(), best_path + '.clf')

        with open(best_path + '_args.txt', 'w') as f:
            for attr, value in sorted(args.__dict__.items()):
                f.write("{}={}\n".format(attr, value))

    return


def train_one(batch, model, opt, args, grad):
    '''
        Train the model on one sampled task.
    '''
    model['ebd'].train()
    model['clf'].train()
    opt.zero_grad()
    
    batch['text'] = batch['text'].to(device)
    batch['label'] = batch['label'].to(device)

    XS = model['ebd'](batch)
    YS = batch['label']

    # if args.classifier == "cesta":
    #     acc, loss, f1, mcc, f1_micro = model['clf'](XS, YS, None, None, authors=batch['authors'])
    # else:
    # Apply the classifier (need to be MLP classifier)
    #acc, loss, f1, mcc = model['clf'](XS, YS, None, None)

    out_XS = model['clf'](XS, None, None, None)
    
    output = out_XS.view(-1, args.n_classes)  # new shape: [32*35, 7]

    # Flatten the target
    target = YS.view(-1)  # new shape: [32*35]

    loss = F.cross_entropy(output, target)
    # acc = BASE.compute_acc(out_XS, YS)
    # f1 = BASE.compute_f1(out_XS, YS)
    # mcc = BASE.compute_mcc(out_XS, YS)
    
    if loss is not None:
        loss.backward()

    if torch.isnan(loss):
        return

    # if args.clip_grad is not None:
    #     nn.utils.clip_grad_value_(grad_param(model, ['ebd', 'clf']), args.clip_grad)
    #     # nn.utils.clip_grad_norm_(grad_param(model, ['ebd', 'clf']), args.clip_grad) #0.5

    grad['clf'].append(get_norm(model['clf']))
    grad['ebd'].append(get_norm(model['ebd']))

    opt.step()


def test(test_data, model, args, verbose=True, target='val', loader=None):
    '''
        Evaluate the model on a bag of sampled tasks. Return the mean accuracy, 
        the weighted f1 score and the matthew correlation coeficient and their
        associated std. (ensure the model used is modified to return the values)
    '''
    model['ebd'].eval()
    model['clf'].eval()

    acc, f1, mcc, f1_micro, trues, preds = [], [], [], [], [], []
    # if loader is None:
    #     loader = DataLoader(SupervisedDataset(test_data, args), batch_size=args.batch_size, num_workers=2, shuffle=False)

    for batch in tqdm(loader, desc=colored('Testing regular on %s' % (target), 'yellow'), total=loader.__len__()):
        #res_acc, res_f1, res_mcc, res_f1_micro, res_pred, res_true = test_one(batch, model, args, out=(target=='test'))
        res_acc, res_f1, res_mcc, res_f1_micro = test_one(batch, model, args, out=(target=='test'))
        acc.append(res_acc)
        f1.append(res_f1)
        mcc.append(res_mcc)
        f1_micro.append(res_f1_micro)
        # trues.extend(res_true.cpu().detach().tolist())
        # preds.extend(res_pred.cpu().detach().tolist())

    acc, f1, mcc, f1_micro = np.array(acc), np.array(f1), np.array(mcc), np.array(f1_micro)

    # if target == 'test' and args.dataset == 'ouitchat_seq':
    #     target_names = ['no emotion', 'anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise']
    #     labels = [0, 1, 2, 3, 4, 5, 6]
    #     print(confusion_matrix(np.array(trues), np.array(preds), labels=labels))
    #     print(classification_report(np.array(trues), np.array(preds), labels=labels, target_names=target_names ) )

    if verbose:
        print("{}, {:s} {:>7.4f} ({:s} {:>7.4f}), {:s} {:>7.4f} ({:s} {:>7.4f}), {:s} {:>7.4f} ({:s} {:>7.4f}), {:s} {:>7.4f} ({:s} {:>7.4f})".format(
                datetime.datetime.now().strftime('%02y/%02m/%02d %H:%M:%S'),
                colored("acc mean", "blue"),
                np.mean(acc),
                colored("std", "blue"),
                np.std(acc),
                colored("f1 mean", "blue"),
                np.mean(f1),
                colored("std", "blue"),
                np.std(f1),
                colored("mcc mean", "blue"),
                np.mean(mcc),
                colored("std", "blue"),
                np.std(mcc),
                colored("f1 micro mean", "blue"),
                np.mean(f1_micro),
                colored("std", "blue"),
                np.std(f1_micro),
                ), flush=True)

        # latex table
        print("{:s} & {:s} & {:>7.4f} \\tiny $\\pm {:>7.4f}$ & {:>7.4f} \\tiny $\\pm {:>7.4f}$ & {:>7.4f} \\tiny $\\pm {:>7.4f}$ & {:>7.4f} \\tiny $\\pm {:>7.4f}$".format(
                args.embedding.replace('_', '\\_'),
                args.classifier.replace('_', '\\_'),
                np.mean(acc),
                np.std(acc),
                np.mean(f1),
                np.std(f1),
                np.mean(mcc),
                np.std(mcc),
                np.mean(f1_micro),
                np.std(f1_micro),
                ), flush=True)
    # if args.classifier == 'cesta':
    #     return np.mean(acc), np.std(acc), np.mean(f1), np.std(f1), np.mean(mcc), np.std(mcc), np.mean(f1_micro), np.std(f1_micro)
    return np.mean(acc), np.std(acc), np.mean(f1), np.std(f1), np.mean(mcc), np.std(mcc), np.mean(f1_micro), np.std(f1_micro)


def test_one(batch, model, args, out=False):
    '''
        Evaluate the model on one sampled task. Return the accuracy.
    '''

    batch['text'] = batch['text'].to(device)
    batch['label'] = batch['label'].to(device)

    # Embedding the document
    XS = model['ebd'](batch)
    YS = batch['label']

    # # Apply the classifier
    # if args.dump and out:
    #     acc, loss, f1, mcc = model['clf'](XS, YS=YS, out=out, XS_ids=batch['ids'])
    # elif out and args.classifier != 'cesta':
    #     acc, loss, f1, mcc, y_pred, y_true = model['clf'](XS, YS=YS, return_preds=True)
    #     return acc, f1, mcc, y_pred, y_true
    # else:
    #     if args.classifier == 'cesta': 
    #         acc, loss, f1, mcc, f1_micro, y_pred, y_true = model['clf'](XS, YS=YS, authors=batch['authors'], return_preds=True)
    #         return acc, f1, mcc, f1_micro, y_pred, y_true
    #     else: acc, loss, f1, mcc = model['clf'](XS, YS=YS)

    out_XS = model['clf'](XS, YS=None)

    output = out_XS.view(-1, args.n_classes)  # new shape: [32*35, 7]
    target = YS.view(-1)  # new shape: [32*35]

    #loss = F.cross_entropy(output, YS)
    acc = BASE.compute_acc(output, target)
    f1 = BASE.compute_f1(output, target)
    mcc = BASE.compute_mcc(output, target)
    micro_f1_noneutral = BASE.compute_f1_micro_noneutral(output, target, labels=args['labels'])
    
    return acc, f1, mcc, micro_f1_noneutral
    
    # if out : #and args.classifier != 'cesta':
    #     acc, loss, f1, mcc, y_pred, y_true = model['clf'](XS, YS=YS, return_preds=True)
    #     return acc, f1, mcc, y_pred, y_true
    # else:
    #     # if args.classifier == 'cesta': 
    #     #     acc, loss, f1, mcc, f1_micro, y_pred, y_true = model['clf'](XS, YS=YS, authors=batch['authors'], return_preds=True)
    #     #     return acc, f1, mcc, f1_micro, y_pred, y_true
    #     #else: acc, loss, f1, mcc = model['clf'](XS, YS=YS)
    #     acc, loss, f1, mcc = model['clf'](XS, YS=YS)

    # return acc, f1, mcc


In [24]:
# ## 예시
# model['ebd'].train()
# model['clf'].train()

# batch_ = train_loader.dataset[0:2]
# batch_['text'] = torch.tensor(batch_['text']).to(device)
# batch_['label'] = torch.tensor(batch_['label']).to(device)

# batch_

In [25]:
# wordebd = WORDEBD(vocab, args.finetune_ebd).to(device)
# batch_['text'] = batch_['text'].to(device)
# batch_['label'] = batch_['label'].to(device)
# ebd_xs_ = wordebd(batch_)
# ebd_xs_.shape

In [26]:
# cnnebd = CNNseq(wordebd, args).to(device)
# batch_['text'] = batch_['text'].to(device)
# batch_['label'] = batch_['label'].to(device)
# cnn_xs_ = cnnebd(batch_)
# cnn_xs_.shape

In [27]:
# xs_ = model['ebd'](batch_)
# ys_ = batch_['label']

In [28]:
# xs_.size(), ys_.size()

In [29]:
# out_xs = model['clf'](xs_, None, None, None)
# out_xs.shape

In [30]:
# output = out_xs.view(-1, 7)  # new shape: [32*35, 7]

# # Flatten the target
# target = ys_.view(-1)  # new shape: [32*35]

# # Compute the loss
# loss = F.cross_entropy(output, target)

# loss

In [17]:
train(train_data, val_data, model, args, loader=train_loader)

23/12/06 18:25:51, Start training supervised


Testing regular on val: 100%|██████████| 32/32 [00:01<00:00, 18.30it/s]8.21it/s]

0.9769531320780516 0.0053124940998657 0.9729722728874199 0.007060956560666433 0.4852064679405341 0.09166620261637358
23/12/06 18:26:16, ep  0, val   acc: 0.9770 ± 0.0053, f1: 0.9730 ± 0.0071, mcc: 0.4852 ± 0.0917, f1 micro: 0.4680 ± 0.1017,train stats ebd_grad: 0.0900, clf_grad: 0.1744


23/12/06 18:26:16, Attempt to save cur best model to /home/hyuns6100/Mental-Heatlh-Care/Result/0
23/12/06 18:26:16, Saved cur best model to /home/hyuns6100/Mental-Heatlh-Care/Result/0


Testing regular on val: 100%|██████████| 32/32 [00:01<00:00, 22.79it/s]6.96it/s]

...


Testing regular on val: 100%|██████████| 32/32 [00:01<00:00, 21.52it/s]6.94it/s]

0.9763671960681677 0.006469304519178751 0.9736552840324121 0.008010762498960175 0.48989775739468316 0.09411799069483878
23/12/06 18:44:53, ep 49, val   acc: 0.9764 ± 0.0065, f1: 0.9737 ± 0.0080, mcc: 0.4899 ± 0.0941, f1 micro: 0.4669 ± 0.1028,train stats ebd_grad: 0.1173, clf_grad: 0.1000



Testing regular on val: 100%|██████████| 32/32 [00:01<00:00, 19.42it/s]7.49it/s]

0.9752232171595097 0.007118401651123753 0.9733954375035826 0.00828162161878278 0.48814614895031555 0.079989059508086
23/12/06 18:45:16, ep 50, val   acc: 0.9752 ± 0.0071, f1: 0.9734 ± 0.0083, mcc: 0.4881 ± 0.0800, f1 micro: 0.4667 ± 0.0894,train stats ebd_grad: 0.1176, clf_grad: 0.1038
23/12/06 18:45:16, End of training. Restore the best weights


23/12/06 18:45:16, Save best model to /home/hyuns6100/Mental-Heatlh-Care/Result/result_text/best


In [18]:
val_acc, val_std, _, _, _, _, val_f1_micro, val_f1_micro_std = test(val_data, model, args, verbose=True, target='val', loader=val_loader)

Testing regular on val: 100%|██████████| 32/32 [00:01<00:00, 21.40it/s]

23/12/06 18:45:31, acc mean  0.9751 (std  0.0070), f1 mean  0.9751 (std  0.0074), mcc mean  0.5559 (std  0.1109), f1 micro mean  0.5410 (std  0.1199)
cnn & mlp &  0.9751 \tiny $\pm  0.0070$ &  0.9751 \tiny $\pm  0.0074$ &  0.5559 \tiny $\pm  0.1109$ &  0.5410 \tiny $\pm  0.1199$


## Test

In [19]:
print( colored('test_data', 'green') )

test_acc, test_std, _, _, _, _, test_f1_micro, test_f1_micro_std, = test(test_data, model, args, target='test', loader=test_loader)

test_data


Testing regular on test: 100%|██████████| 32/32 [00:01<00:00, 23.15it/s]

23/12/06 18:45:45, acc mean  0.9637 (std  0.0112), f1 mean  0.9613 (std  0.0132), mcc mean  0.4940 (std  0.0956), f1 micro mean  0.4837 (std  0.0988)
cnn & mlp &  0.9637 \tiny $\pm  0.0112$ &  0.9613 \tiny $\pm  0.0132$ &  0.4940 \tiny $\pm  0.0956$ &  0.4837 \tiny $\pm  0.0988$


In [20]:
if args.best_result_path:
    directory = args.best_result_path[:args.best_result_path.rfind("/")]
    if not os.path.exists(directory):
        os.makedirs(directory)

    result = {
        "test_acc": test_acc,
        "test_std": test_std,
        "val_acc": val_acc,
        "val_std": val_std,
        "test_f1_micro": test_f1_micro,
        "test_f1_micro_std": test_f1_micro_std,
        "val_f1_micro": val_f1_micro,
        "val_f1_micro_std": val_f1_micro_std
    }

    for attr, value in sorted(args.__dict__.items()):
        result[attr] = value

    with open(args.best_result_path, "wb") as f:
        pickle.dump(result, f, pickle.HIGHEST_PROTOCOL)

In [21]:
print(result)

{'test_acc': 0.9636718761175871, 'test_std': 0.011181758962263125, 'val_acc': 0.9750837124884129, 'val_std': 0.006978293767178298, 'test_f1_micro': 0.4836933431523283, 'test_f1_micro_std': 0.09875537229629391, 'val_f1_micro': 0.5409813460077257, 'val_f1_micro_std': 0.11992616569879222, 'authors': False, 'batch_size': 32, 'best_result_path': '/home/hyuns6100/Mental-Heatlh-Care/Result/best_results.pkl', 'classifier': 'mlp', 'cnn_filter_sizes': [3, 4, 5], 'cnn_num_filters': 100, 'context_size': 35, 'convmode': 'seq', 'data_path': '/home/hyuns6100/Mental-Heatlh-Care/data/dailydialog_conv35seq_splits.json', 'dropout': 0.1, 'embedding': 'cnn', 'finetune_ebd': False, 'labels': [1, 2, 3, 4, 5, 6], 'lr': 0.001, 'maxtokens': 30, 'mlp_hidden': [300, 300], 'n_classes': 7, 'n_test_class': 7, 'n_train_class': 7, 'n_val_class': 7, 'patience': 30, 'patience_metric': 'f1_micro', 'result_path': '/home/hyuns6100/Mental-Heatlh-Care/Result/', 'result_text_path': '/home/hyuns6100/Mental-Heatlh-Care/Result/r

In [ ]:
# def eval_torch(audio_path,checkpoint_path):
#     torch_model = resnet34().cuda()
#     load_model = torch.load(checkpoint_path)
#     torch_model.load_state_dict(load_model)
#     torch_model.eval()
    

#     paths = glob.glob(audio_path+"/*")
#     count = 0
#     with torch.no_grad():
#         count = 0
#         for input_path in tqdm(paths):
            
#             label_ = int(input_path.split('-')[2]) - 1
#             input_data = preprocess_audio(input_path)
#             input_data = np.transpose(input_data, axes=[2, 0, 1])
#             input_data = np.expand_dims(input_data, axis=0)
#             input_data = torch.FloatTensor(input_data).cuda()
#             outputs = torch_model(input_data)
                
#             pred = torch.argmax(outputs.cpu())

#             if label_ == pred:
#                 count += 1
            
#     print("=======================")
#     print(">> Pytorch result")
#     print(f"Acc: {count/len(paths)}")
#     print("=======================")


## PyTorch to ONNX

### onnx 변환 전 성능 확인

In [57]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#best_path = "/home/hyuns6100/Mental-Heatlh-Care/Result/"
best_path = "/home/hyuns6100/Mental-Heatlh-Care/Result/result_text/best"

# 모델 state path
ebd_model_path = best_path + '.ebd'
clf_model_path = best_path + '.clf'

ebd_model_params = torch.load(ebd_model_path, map_location=device)
clf_model_params = torch.load(clf_model_path, map_location=device)

# 모델 정의 및 불러온 파라미터 설정
try_model = {}
wordebd = WORDEBD(vocab, finetune_ebd=False)
ebd = CNNseq(wordebd, args).to(device)
try_model['ebd'] = ebd

clf = MLPseq(try_model["ebd"].ebd_dim, args).to(device)
try_model['clf'] = clf

try_model['ebd'].load_state_dict(ebd_model_params)
try_model['clf'].load_state_dict(clf_model_params)

try_model['ebd'].eval()
try_model['clf'].eval()

MLPseq(
  (mlp): Sequential(
    (0): Dropout(p=0.1, inplace=False)
    (1): Linear(in_features=300, out_features=300, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.1, inplace=False)
    (4): Linear(in_features=300, out_features=300, bias=True)
  )
  (out): Linear(in_features=300, out_features=7, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [30]:
test_acc, test_std, _, _, _, _, test_f1_micro, test_f1_micro_std, = test(test_data, try_model, args, target='test', loader=test_loader)

Testing regular on test: 100%|██████████| 32/32 [00:01<00:00, 25.58it/s]

23/12/06 18:55:06, acc mean  0.9637 (std  0.0112), f1 mean  0.9613 (std  0.0132), mcc mean  0.4940 (std  0.0956), f1 micro mean  0.4837 (std  0.0988)
cnn & mlp &  0.9637 \tiny $\pm  0.0112$ &  0.9613 \tiny $\pm  0.0132$ &  0.4940 \tiny $\pm  0.0956$ &  0.4837 \tiny $\pm  0.0988$


In [24]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# #best_path = "/home/hyuns6100/Mental-Heatlh-Care/Result/"
# best_path = "/home/hyuns6100/Mental-Heatlh-Care/Result/result_text/best"

# # 모델 state path
# ebd_model_path = best_path + '.ebd'
# clf_model_path = best_path + '.clf'

# ebd_model_params = torch.load(ebd_model_path, map_location=device)
# clf_model_params = torch.load(clf_model_path, map_location=device)

# # 모델 정의 및 불러온 파라미터 설정
# model = {}
# wordebd = WORDEBD(vocab, finetune_ebd=False)
# ebd = CNNseq(wordebd, args).to(device)
# model['ebd'] = ebd

# clf = MLPseq(model["ebd"].ebd_dim, args).to(device)
# model['clf'] = clf

# model['ebd'].load_state_dict(ebd_model_params)
# model['clf'].load_state_dict(clf_model_params)

# model['ebd'].eval()
# model['clf'].eval()

# calibration data 
batch = test_loader.dataset[0]
batch['text'] = torch.tensor(batch['text']).unsqueeze(0).to(device)

XS = try_model['ebd'](batch)
# XS_np = XS.detach().cpu().numpy()

# # save to .npy
# np.save('/home/hyuns6100/Mental-Heatlh-Care/onnx/emotion_calib_1', XS_np)

# convert to onnx
torch.onnx.export(try_model['clf'], XS, "/home/hyuns6100/Mental-Heatlh-Care/onnx/emo_clf_onnx.onxx")

In [25]:
import numpy as np

test_np_data = np.load('/home/hyuns6100/Mental-Heatlh-Care/onnx/emotion_test_data.npy')
test_np_label = np.load('/home/hyuns6100/Mental-Heatlh-Care/onnx/emotion_test_label.npy')

batch = torch.tensor(test_np_data).to(device)
batch_label = torch.tensor(test_np_label).to(device)

In [29]:
# test_one 함수
s = time.time()
out_XS = try_model['clf'](batch, YS=None)
print(time.time() - s)

output = out_XS.view(-1, args.n_classes)  # new shape: [32*35, 7] (batch_size=32)
target = batch_label.view(-1)

acc = BASE.compute_acc(output, target)
f1 = BASE.compute_f1(output, target)
mcc = BASE.compute_mcc(output, target)
micro_f1_noneutral = BASE.compute_f1_micro_noneutral(output, target, labels=args['labels'])

acc, f1, mcc, micro_f1_noneutral

0.1465756893157959


(0.9594571590423584, 0.9396051476148122, 0.0, 0.0)

In [56]:
(torch.argmax(output, dim=1) == target).float().sum() / len(output)

tensor(0.9595, device='cuda:0')

## Save to npy 

In [35]:
try_model['ebd'].eval()
try_model['clf'].eval()

MLPseq(
  (mlp): Sequential(
    (0): Dropout(p=0.1, inplace=False)
    (1): Linear(in_features=300, out_features=300, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.1, inplace=False)
    (4): Linear(in_features=300, out_features=300, bias=True)
  )
  (out): Linear(in_features=300, out_features=7, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [37]:
batch = test_loader.dataset[:]
batch['text'] = torch.tensor(batch['text']).to(device)

YS = torch.tensor(batch['label']).to(device)
XS = try_model['ebd'](batch)
XS_np = XS.detach().cpu().numpy()
YS_np = YS.detach().cpu().numpy()

# save to .npy
np.save('/home/hyuns6100/Mental-Heatlh-Care/onnx/emotion_test_data', XS_np)
np.save('/home/hyuns6100/Mental-Heatlh-Care/onnx/emotion_test_label', YS_np)

# 성능 확인 마지막!! 

In [75]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#best_path = "/home/hyuns6100/Mental-Heatlh-Care/Result/"
best_path = "/home/hyuns6100/Mental-Heatlh-Care/Result/result_text/best"

# 모델 state path
ebd_model_path = best_path + '.ebd'
clf_model_path = best_path + '.clf'

ebd_model_params = torch.load(ebd_model_path, map_location=device)
clf_model_params = torch.load(clf_model_path, map_location=device)

# 모델 정의 및 불러온 파라미터 설정
try_model = {}
wordebd = WORDEBD(vocab, finetune_ebd=False)
ebd = CNNseq(wordebd, args).to(device)
try_model['ebd'] = ebd

clf = MLPseq(try_model["ebd"].ebd_dim, args).to(device)
try_model['clf'] = clf

try_model['ebd'].load_state_dict(ebd_model_params)
try_model['clf'].load_state_dict(clf_model_params)

try_model['ebd'].eval()
try_model['clf'].eval()

MLPseq(
  (mlp): Sequential(
    (0): Dropout(p=0.1, inplace=False)
    (1): Linear(in_features=300, out_features=300, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.1, inplace=False)
    (4): Linear(in_features=300, out_features=300, bias=True)
  )
  (out): Linear(in_features=300, out_features=7, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [76]:
# try_model: best 모델 파라미터 load 
# XS: test_loader 에 저장된 모든 데이터 (1000개) 를 try_model로 임베딩한 input 데이터


# 임베딩 변환 후 & npy 저장 전
#### 데이터 생성
batch = test_loader.dataset[:]
batch['text'] = torch.tensor(batch['text']).to(device)

YS = torch.tensor(batch['label']).to(device) 
XS = try_model['ebd'](batch)


#### model 평가
out_XS = try_model['clf'](XS, YS=None)

output = out_XS.view(-1, args.n_classes)  # new shape: [32*35, 7]
target = YS.view(-1)

acc = BASE.compute_acc(output, target)
f1 = BASE.compute_f1(output, target)
mcc = BASE.compute_mcc(output, target)
micro_f1_noneutral = BASE.compute_f1_micro_noneutral(output, target, labels=args['labels'])

acc, f1, mcc, micro_f1_noneutral

(0.9634857177734375,
 0.9612830286849585,
 0.4899312517676465,
 0.4828393135725429)

In [77]:
# 임베딩 변환 후 & npy 저장 및 다시 load

test_np_data = np.load('/home/hyuns6100/Mental-Heatlh-Care/onnx/emotion_test_data.npy')
test_np_label = np.load('/home/hyuns6100/Mental-Heatlh-Care/onnx/emotion_test_label.npy')

test = torch.tensor(test_np_data).to(device)
label = torch.tensor(test_np_label).to(device)

#### model 평가
import time
s = time.time()
out_XS = try_model['clf'](test, YS=None)
print(time.time() - s)


output = out_XS.view(-1, args.n_classes)  # new shape: [32*35, 7]
#label = YS.view(-1)
label = label.view(-1)

acc = BASE.compute_acc(output, label)
f1 = BASE.compute_f1(output, label)
mcc = BASE.compute_mcc(output, label)
micro_f1_noneutral = BASE.compute_f1_micro_noneutral(output, label, labels=args['labels'])

acc, f1, mcc, micro_f1_noneutral

0.0007719993591308594


(0.9634857177734375,
 0.9612830286849585,
 0.4899312517676465,
 0.4828393135725429)

## onnx -> pytorch

In [72]:
# import onnx

# model = onnx.load("/home/hyuns6100/Mental-Heatlh-Care/onnx/emotion_calib_1.onxx")

In [79]:
import onnx
from onnx2pytorch import ConvertModel
import torch
# import time

# # Load the ONNX model
# onnx_model = onnx.load("/home/hyuns6100/Mental-Heatlh-Care/onnx/emotion_calib_1.onxx")

# # Convert to PyTorch model
# pytorch_model = ConvertModel(onnx_model)

ImportError: cannot import name '_LazyBatchNorm' from 'torch.nn.modules.batchnorm' (/home/hyuns6100/anaconda3/envs/MentalHealth/lib/python3.8/site-packages/torch/nn/modules/batchnorm.py)

In [90]:
import numpy as np

test_np_data = np.load('/home/hyuns6100/Mental-Heatlh-Care/onnx/emotion_test_data.npy')
test_np_label = np.load('/home/hyuns6100/Mental-Heatlh-Care/onnx/emotion_test_label.npy')

# test = torch.tensor(test_np_data).to(device)
# label = torch.tensor(test_np_label).to(device)



In [84]:
# %pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 77.5 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [92]:
import onnx, onnxruntime

onnx_path = "/home/hyuns6100/Mental-Heatlh-Care/onnx/emo_clf_onnx.onxx"
onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model)

providers = ["TensorrtExecutionProvider", "CUDAExecutionProvider"]
session = onnxruntime.InferenceSession(onnx_path, providers=providers)


/home/hyuns6100/anaconda3/envs/MentalHealth/lib/python3.8/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'TensorrtExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(
/home/hyuns6100/anaconda3/envs/MentalHealth/lib/python3.8/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


In [97]:
for td in test_np_data:
    outputs = session.run(None, {"input.1": td})

InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Invalid rank for input: input.1 Got: 2 Expected: 3 Please fix either the inputs or the model.